<a href="https://colab.research.google.com/github/AndyZhang127/temp/blob/main/Dissertation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Loading essential modules

In [ ]:
#Loading Essential Packages

import pandas as pd
import numpy as np
import math


# 2.Defining necessary functions

## Part A: Producing missing value randomly

In [ ]:
#Producing the missing data randomly
def missingRate_Data(df,columns,missing_rate=0):
  Source_missing = df.sample(frac=missing_rate)
  Source_missing_re=(Source_missing[columns] * 0).replace(0,np.nan)
  Source_missing[columns]=Source_missing_re[columns]
  index = Source_missing.index
  non_missing_df=df.query('index not in @index')
  Source_missing_final=pd.concat([non_missing_df,Source_missing])

  return Source_missing_final,index,Source_missing,Source_missing,non_missing_df



## Part B: Defining basic imputting methods (mean,mode,median)

In [ ]:
# Define replace methods

def mean_replace(df,col):
  mean=df.agg({col:'mean'}).head(1)[0]
  df=df.fillna(mean)
  return df

def mode_replace(df,col):
  mode=df[col].mode()[0]
  df=df.fillna(mode)
  return df

def median_replace(df,col):
  median=df.agg({col:'median'}).head(1)[0]
  df=df.fillna(median)
  return df

## Part C: Defining Algrithm for immputation  (SVR,RF,LSTM,RNN,MLP,GCU,FBPROPHET)

In [ ]:
class SVRModel(object):

    def __init__(self, C=1.0, kernel='rbf', epsilon=0.2, shrinking=True):
        self.C = C
        self.kernel = kernel
        self.shrinking = shrinking
        self.epsilon = epsilon
        self.buildModel()

    def buildModel(self,):
        self.model = SVR(C=self.C, cache_size=200, coef0=0.0, degree=3, epsilon=self.epsilon, gamma='auto',
            kernel=self.kernel, max_iter=-1, shrinking=self.shrinking, tol=0.001, verbose=False)

    def train(self, trainX, trainY):
        self.model.fit(trainX, trainY)

    def predict(self, testX):
        pred = self.model.predict(testX)
        return pred

class MLP_Model(object):

    def __init__(self, inputDim, hiddenNum, outputDim, lr):

        self.inputDim = inputDim
        self.hiddenNum = hiddenNum
        self.outputDim = outputDim
        self.opt = keras.optimizers.RMSprop(lr=lr, rho=0.9, epsilon=1e-06)
        self.buildModel()

    def buildModel(self):

        self.model = Sequential()
        self.model.add(Dense(self.hiddenNum, input_dim=self.inputDim, activation='relu'))
        self.model.add(Dense(1))
        self.model.compile(loss='mean_squared_error', optimizer=self.opt)

    def train(self, trainX, trainY, epoch, batchSize):
        self.model.fit(trainX, trainY, epochs=epoch, batch_size=batchSize, verbose=1)

    def predict(self, testX):
        pred = self.model.predict(testX)
        return pred

class RNNsModel(object):
    def __init__(self, inputDim, hiddenNum, outputDim, unit, lr):

        self.inputDim = inputDim
        self.hiddenNum = hiddenNum
        self.outputDim = outputDim
        self.opt = optimizers.RMSprop(lr=lr, rho=0.9, epsilon=1e-06)
        self.buildModel(unit)

    def buildModel(self, unit="GRU"):

        self.model = Sequential()
        if unit == "GRU":
            self.model.add(GRU(self.hiddenNum, input_shape=(None, self.inputDim)))
        elif unit == "LSTM":
            self.model.add(LSTM(self.hiddenNum, input_shape=(None, self.inputDim)))
        elif unit == "RNN":
            self.model.add(SimpleRNN(self.hiddenNum, input_shape=(None, self.inputDim)))
        self.model.add(Dense(self.outputDim))
        self.model.compile(loss='mean_squared_error', optimizer=self.opt, metrics=["mean_absolute_percentage_error"])

    def train(self, trainX, trainY, epoch, batchSize):
        self.model.fit(trainX, trainY, epochs=epoch, batch_size=batchSize, verbose=1, validation_split=0.0)

    def predict(self,testX):
        pred = self.model.predict(testX)
        return pred

## Part D: Defining validation methods

In [ ]:
# Define validation methods

def MSE(actual,predicted):
  MSE = np.square(np.subtract(actual,predicted)).mean() 
  MSE = math.sqrt(MSE)
  return MSE

def RMSE(actual,predicted):
  MSE = np.square(np.subtract(actual,predicted)).mean() 
  MSE = math.sqrt(MSE)
  RMSE = math.sqrt(MSE)
  return RMSE

def MAD(actual,predicted):
  ''''''
  

## Part E: Defining visualization methods

In [ ]:
# print all validation result

def result_validation(Columns,method='MSE'):
    result={}
    for  col in Columns:
      Temp_actual=Source_Temp[col].tolist()
      Temp_predicted=Temp_missing_final_1[col].tolist()
      result[col]=MSE(Temp_actual,Temp_predicted)




# 3.Deploy different approches on datasets

In [ ]:
#Loading Source Data
source_file=pd.read_csv('/content/Occupancy_Estimation.csv')
source_file

In [ ]:
#Pickup Temp Data
Temp_Columns=['Date','Time','S1_Temp','S2_Temp','S3_Temp','S4_Temp']
Source_Temp=source_file[Temp_Columns]

In [ ]:
Source_Temp

In [ ]:
#Producing the missing data randomly
columns=['S1_Temp','S2_Temp','S3_Temp','S4_Temp']
Temp_missing_final_1,Temp_missing_index_1,Source_missing_1,Source_missing_re_1,non_missing_df_1=missingRate_Data(Source_Temp,columns,missing_rate=0.05)
Temp_missing_final_2,Temp_missing_index_2,Source_missing_2,Source_missing_re_2,non_missing_df_2=missingRate_Data(Source_Temp,columns,missing_rate=0.10)
Temp_missing_final_3,Temp_missing_index_3,Source_missing_3,Source_missing_re_3,non_missing_df_3=missingRate_Data(Source_Temp,columns,missing_rate=0.15)
Temp_missing_final_4,Temp_missing_index_4,Source_missing_4,Source_missing_re_4,non_missing_df_4=missingRate_Data(Source_Temp,columns,missing_rate=0.20)
Temp_missing_final_5,Temp_missing_index_5,Source_missing_5,Source_missing_re_5,non_missing_df_5=missingRate_Data(Source_Temp,columns,missing_rate=0.30)



In [ ]:
index=Temp_missing_index_1
non_missing_df=Source_Temp.query('index not in @index')
missing_df=Source_Temp.query('index in @index')

In [ ]:
x_train=non_missing_df['datetime','S1_Temp']
y_train=missing_df['S1_Temp']
